In [1]:
!pip install pyspark
!apt update
!apt install openjdk-8-jdk-headless -qq
import os
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=b75ebe5abe06e290cb6fab090223b6e92bc56d5e1ae489bc2227bc22954bc08a
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,71

In [2]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)
df = sqlContext.read.csv('transactions_train.csv', header=True, inferSchema=True)
transactions_rdd = df.rdd

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
from math import floor
def redondeo(numero, decimal):
  ajuste = 10**decimal
  return floor(numero * ajuste) / ajuste

In [ ]:
transactions_rdd.first()

Row(_c0=0, Id=2987000, target=0, timestamp=86400, ammount=68.5, product_category='W', feat_card_1=13926, feat_card_2=None, feat_card_3=150.0, feat_card_4='discover', feat_card_5=142.0, feat_card_6='credit', addr1=315.0, addr2=87.0, dist1=19.0, dist2=None, payer_email_domain=None, receiver_email_domain=None, A1=1.0, A2=1.0, A3=0.0, A4=0.0, A5=0.0, A6=1.0, A7=0.0, A8=0.0, A9=1.0, A10=0.0, A11=2.0, A12=0.0, A13=1.0, A14=1.0, B1=14.0, B2=None, B3=13.0, B4=None, B5=None, B6=None, B7=None, B8=None, B9=None, B10=13.0, B11=13.0, B12=None, B13=None, B14=None, B15=0.0, C1='T', C2='T', C3='T', C4='M2', C5='F', C6='T', C7=None, C8=None, C9=None, D1=1.0, D2=1.0, D3=1.0, D4=1.0, D5=1.0, D6=1.0, D7=1.0, D8=1.0, D9=1.0, D10=0.0, D11=0.0, D12=1.0, D13=1.0, D14=1.0, D15=0.0, D16=0.0, D17=0.0, D18=0.0, D19=1.0, D20=1.0, D21=0.0, D22=0.0, D23=1.0, D24=1.0, D25=1.0, D26=1.0, D27=0.0, D28=0.0, D29=0.0, D30=0.0, D31=0.0, D32=0.0, D33=0.0, D34=0.0, D35=None, D36=None, D37=None, D38=None, D39=None, D40=None, D

Porcentaje de fraude

In [ ]:
transactions_rdd.map(lambda x: x.target).distinct().collect()

[0, 1]

In [ ]:
redondeo(transactions_rdd.filter(lambda x: x.target == 1).count() * 100 / transactions_rdd.count(), 2)

3.5

Monto respecto de los productos

In [4]:
productos = transactions_rdd.map(lambda x: ((x.product_category, x.target), x.ammount)).reduceByKey(lambda a,b: a + b).map(lambda x: (x[0][0], (x[0][1], x[1])))

In [5]:
productos_total = productos.map(lambda x: (x[0], x[1][1])).reduceByKey(lambda a,b: a + b)

In [6]:
productos = productos.join(productos_total).map(lambda x: (x[0], x[1][0][0], redondeo(x[1][0][1], 2), redondeo(x[1][0][1] * 100 / x[1][1], 2))).cache()

In [ ]:
productos.collect()

[('H', 0, 1986435.0, 89.91),
 ('H', 1, 222872.0, 10.08),
 ('R', 0, 5424125.0, 94.86),
 ('R', 1, 293550.0, 5.13),
 ('S', 0, 427847.1, 92.01),
 ('S', 1, 37121.0, 7.98),
 ('W', 1, 1730240.03, 3.05),
 ('W', 0, 54909721.53, 96.94),
 ('C', 0, 2227829.57, 87.08),
 ('C', 1, 330453.3, 12.91)]

Cantidad de monto respecto de los productos fraudulentos

In [ ]:
productos.filter(lambda x: x[1] == 1).map(lambda x: (x[0], x[2])).sortBy(lambda x: -x[1]).collect()

[('W', 1730240.03),
 ('C', 330453.3),
 ('R', 293550.0),
 ('H', 222872.0),
 ('S', 37121.0)]

Cantidad de monto respecto de los productos legales

In [ ]:
productos.filter(lambda x: x[1] == 0).map(lambda x: (x[0], x[2])).sortBy(lambda x: -x[1]).collect()

[('W', 54909721.53),
 ('R', 5424125.0),
 ('C', 2227829.57),
 ('H', 1986435.0),
 ('S', 427847.1)]

Porcentaje de monto respecto de los productos fraudulentos

In [ ]:
productos.filter(lambda x: x[1] == 1).map(lambda x: (x[0], x[3])).sortBy(lambda x: -x[1]).collect()

[('C', 12.91), ('H', 10.08), ('S', 7.98), ('R', 5.13), ('W', 3.05)]

Porcentaje de monto respecto de los productos legales

In [ ]:
productos.filter(lambda x: x[1] == 0).map(lambda x: (x[0], x[3])).sortBy(lambda x: -x[1]).collect()

[('W', 96.94), ('R', 94.86), ('S', 92.01), ('H', 89.91), ('C', 87.08)]

Fraude segun el tiempo, analizando cantidad y monto

In [8]:
tiempo_fraudulento = transactions_rdd.filter(lambda x: x.target == 1).cache()

In [7]:
def segundos(numero):
  return numero - 86400

def analisis(numero, divisor):
  flotante = (numero - 86400) / divisor
  entero = (numero - 86400) // divisor
  if flotante > entero:
    return entero + 1
  else:
    return entero

def minutos(numero):
  return analisis(numero, 60)

def horas(numero):
  return analisis(numero, 3600)

def dias(numero):
  return analisis(numero, 86400)

def semanas(numero):
  return analisis(numero, 604800)

def meses(numero):
  return analisis(numero, 2419200)

def porcentaje_tupla(tupla, total):
  return (tupla[0], redondeo(tupla[1], 2), redondeo(tupla[1] * 100 / total, 2))

def porcentaje_lista(lista, total):
  nueva_lista = []
  for elemento in lista:
    nueva_lista.append((elemento[0], redondeo(elemento[1], 2), redondeo(elemento[1] * 100 / total, 4)))
  return nueva_lista

Cantidad respecto del tiempo

Cantidad total respecto de segundos

In [10]:
cantidad_total = tiempo_fraudulento.count()
(tiempo_fraudulento.map(lambda x: segundos(x.timestamp)).reduce(lambda a,b: a if a > b else b), cantidad_total, 100)

(13064705, 17580, 100)

Top ten cantidad respecto de minutos

In [ ]:
porcentaje_lista(tiempo_fraudulento.map(lambda x: (minutos(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).takeOrdered(10, lambda x: -x[1]), cantidad_total)

[(139695, 6.0, 0.0341),
 (136426, 6.0, 0.0341),
 (73412, 5.0, 0.0284),
 (148153, 5.0, 0.0284),
 (96611, 4.0, 0.0227),
 (60687, 4.0, 0.0227),
 (27508, 4.0, 0.0227),
 (176918, 4.0, 0.0227),
 (89024, 4.0, 0.0227),
 (140574, 4.0, 0.0227)]

Last ten cantidad respecto de minutos

In [ ]:
porcentaje_lista(tiempo_fraudulento.map(lambda x: (minutos(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).takeOrdered(10, lambda x: x[1]), cantidad_total)

[(100, 1.0, 0.0056),
 (110, 1.0, 0.0056),
 (220, 1.0, 0.0056),
 (490, 1.0, 0.0056),
 (645, 1.0, 0.0056),
 (665, 1.0, 0.0056),
 (835, 1.0, 0.0056),
 (865, 1.0, 0.0056),
 (925, 1.0, 0.0056),
 (1000, 1.0, 0.0056)]

Cantidad de minutos, cantidad de fraudes y promedio

In [34]:
porcentaje_lista(tiempo_fraudulento.map(lambda x: (minutos(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).map(lambda x: (x[1], 1))\
.reduceByKey(lambda a,b: a + b).map(lambda x: (x[1], x[0])).sortBy(lambda x: x[1]).collect(), cantidad_total)

[(15136, 1.0, 0.0056),
 (1097, 2.0, 0.0113),
 (68, 3.0, 0.017),
 (6, 4.0, 0.0227),
 (2, 5.0, 0.0284),
 (2, 6.0, 0.0341)]

Top ten cantidad respecto de horas

In [ ]:
porcentaje_lista(tiempo_fraudulento.map(lambda x: (horas(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).takeOrdered(10, lambda x: -x[1]), cantidad_total)

[(1529, 42.0, 0.2389),
 (1344, 36.0, 0.2047),
 (1530, 33.0, 0.1877),
 (2790, 32.0, 0.182),
 (2781, 32.0, 0.182),
 (2949, 31.0, 0.1763),
 (1026, 27.0, 0.1535),
 (1484, 26.0, 0.1478),
 (430, 25.0, 0.1422),
 (1531, 25.0, 0.1422)]

Last ten cantidad respecto de horas

In [ ]:
porcentaje_lista(tiempo_fraudulento.map(lambda x: (horas(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).takeOrdered(10, lambda x: x[1]), cantidad_total)

[(10, 1.0, 0.0056),
 (110, 1.0, 0.0056),
 (135, 1.0, 0.0056),
 (180, 1.0, 0.0056),
 (185, 1.0, 0.0056),
 (200, 1.0, 0.0056),
 (205, 1.0, 0.0056),
 (250, 1.0, 0.0056),
 (270, 1.0, 0.0056),
 (275, 1.0, 0.0056)]

Cantidad de horas, cantidad de fraudes y promedio

In [35]:
porcentaje_lista(tiempo_fraudulento.map(lambda x: (horas(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).map(lambda x: (x[1], 1))\
.reduceByKey(lambda a,b: a + b).map(lambda x: (x[1], x[0])).sortBy(lambda x: x[1]).collect(), cantidad_total)

[(419, 1.0, 0.0056),
 (399, 2.0, 0.0113),
 (349, 3.0, 0.017),
 (308, 4.0, 0.0227),
 (277, 5.0, 0.0284),
 (252, 6.0, 0.0341),
 (209, 7.0, 0.0398),
 (189, 8.0, 0.0455),
 (134, 9.0, 0.0511),
 (105, 10.0, 0.0568),
 (88, 11.0, 0.0625),
 (72, 12.0, 0.0682),
 (64, 13.0, 0.0739),
 (53, 14.0, 0.0796),
 (21, 15.0, 0.0853),
 (38, 16.0, 0.091),
 (26, 17.0, 0.0967),
 (11, 18.0, 0.1023),
 (7, 19.0, 0.108),
 (9, 20.0, 0.1137),
 (4, 21.0, 0.1194),
 (5, 22.0, 0.1251),
 (1, 23.0, 0.1308),
 (2, 24.0, 0.1365),
 (6, 25.0, 0.1422),
 (1, 26.0, 0.1478),
 (1, 27.0, 0.1535),
 (1, 31.0, 0.1763),
 (2, 32.0, 0.182),
 (1, 33.0, 0.1877),
 (1, 36.0, 0.2047),
 (1, 42.0, 0.2389)]

Cantidad respecto de días

In [ ]:
porcentaje_lista(sorted(tiempo_fraudulento.map(lambda x: (dias(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).collect()), cantidad_total)

[(1, 112.0, 0.637),
 (2, 123.0, 0.6996),
 (3, 92.0, 0.5233),
 (4, 115.0, 0.6541),
 (5, 127.0, 0.7224),
 (6, 99.0, 0.5631),
 (7, 136.0, 0.7736),
 (8, 93.0, 0.529),
 (9, 119.0, 0.6769),
 (10, 90.0, 0.5119),
 (11, 123.0, 0.6996),
 (12, 84.0, 0.4778),
 (13, 82.0, 0.4664),
 (14, 126.0, 0.7167),
 (15, 143.0, 0.8134),
 (16, 70.0, 0.3981),
 (17, 126.0, 0.7167),
 (18, 161.0, 0.9158),
 (19, 118.0, 0.6712),
 (20, 135.0, 0.7679),
 (21, 116.0, 0.6598),
 (22, 147.0, 0.8361),
 (23, 106.0, 0.6029),
 (24, 76.0, 0.4323),
 (25, 63.0, 0.3583),
 (26, 82.0, 0.4664),
 (27, 100.0, 0.5688),
 (28, 150.0, 0.8532),
 (29, 129.0, 0.7337),
 (30, 158.0, 0.8987),
 (31, 149.0, 0.8475),
 (32, 51.0, 0.2901),
 (33, 123.0, 0.6996),
 (34, 159.0, 0.9044),
 (35, 120.0, 0.6825),
 (36, 135.0, 0.7679),
 (37, 126.0, 0.7167),
 (38, 110.0, 0.6257),
 (39, 117.0, 0.6655),
 (40, 109.0, 0.62),
 (41, 121.0, 0.6882),
 (42, 85.0, 0.4835),
 (43, 160.0, 0.9101),
 (44, 119.0, 0.6769),
 (45, 100.0, 0.5688),
 (46, 117.0, 0.6655),
 (47, 91.0, 0

Top ten cantidad respecto de días

In [ ]:
porcentaje_lista(tiempo_fraudulento.map(lambda x: (dias(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).takeOrdered(10, lambda x: -x[1]), cantidad_total)

[(64, 234.0, 1.331),
 (117, 194.0, 1.1035),
 (85, 187.0, 1.0637),
 (65, 183.0, 1.0409),
 (120, 181.0, 1.0295),
 (91, 180.0, 1.0238),
 (50, 175.0, 0.9954),
 (59, 168.0, 0.9556),
 (62, 165.0, 0.9385),
 (116, 163.0, 0.9271)]

Last ten cantidad respecto de días

In [ ]:
porcentaje_lista(tiempo_fraudulento.map(lambda x: (dias(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).takeOrdered(10, lambda x: x[1]), cantidad_total)

[(152, 31.0, 0.1763),
 (32, 51.0, 0.2901),
 (139, 61.0, 0.3469),
 (25, 63.0, 0.3583),
 (122, 64.0, 0.364),
 (80, 66.0, 0.3754),
 (149, 66.0, 0.3754),
 (145, 67.0, 0.3811),
 (101, 68.0, 0.3868),
 (150, 69.0, 0.3924)]

Cantidad respecto de semanas

In [ ]:
porcentaje_lista(sorted(tiempo_fraudulento.map(lambda x: (semanas(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).collect()), cantidad_total)

[(1, 804.0, 4.5733),
 (2, 717.0, 4.0784),
 (3, 869.0, 4.9431),
 (4, 724.0, 4.1183),
 (5, 889.0, 5.0568),
 (6, 803.0, 4.5676),
 (7, 823.0, 4.6814),
 (8, 862.0, 4.9032),
 (9, 927.0, 5.273),
 (10, 996.0, 5.6655),
 (11, 880.0, 5.0056),
 (12, 676.0, 3.8452),
 (13, 912.0, 5.1877),
 (14, 884.0, 5.0284),
 (15, 718.0, 4.0841),
 (16, 866.0, 4.926),
 (17, 1069.0, 6.0807),
 (18, 833.0, 4.7383),
 (19, 727.0, 4.1353),
 (20, 611.0, 3.4755),
 (21, 639.0, 3.6348),
 (22, 351.0, 1.9965)]

Top ten cantidad respecto de semanas

In [ ]:
porcentaje_lista(tiempo_fraudulento.map(lambda x: (semanas(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).takeOrdered(10, lambda x: -x[1]), cantidad_total)

[(17, 1069.0, 6.0807),
 (10, 996.0, 5.6655),
 (9, 927.0, 5.273),
 (13, 912.0, 5.1877),
 (5, 889.0, 5.0568),
 (14, 884.0, 5.0284),
 (11, 880.0, 5.0056),
 (3, 869.0, 4.9431),
 (16, 866.0, 4.926),
 (8, 862.0, 4.9032)]

Last ten cantidad respecto de semanas

In [ ]:
porcentaje_lista(tiempo_fraudulento.map(lambda x: (semanas(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).takeOrdered(10, lambda x: x[1]), cantidad_total)

[(22, 351.0, 1.9965),
 (20, 611.0, 3.4755),
 (21, 639.0, 3.6348),
 (12, 676.0, 3.8452),
 (2, 717.0, 4.0784),
 (15, 718.0, 4.0841),
 (4, 724.0, 4.1183),
 (19, 727.0, 4.1353),
 (6, 803.0, 4.5676),
 (1, 804.0, 4.5733)]

Cantidad respecto de meses

In [ ]:
porcentaje_lista(sorted(tiempo_fraudulento.map(lambda x: (meses(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).collect()), cantidad_total)

[(1, 3114.0, 17.7133),
 (2, 3377.0, 19.2093),
 (3, 3479.0, 19.7895),
 (4, 3380.0, 19.2263),
 (5, 3240.0, 18.43),
 (6, 990.0, 5.6313)]

In [ ]:
porcentaje_tupla(tiempo_fraudulento.map(lambda x: (meses(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).reduce(lambda a,b: a if a[1] > b[1] else b), cantidad_total)

(3, 3479.0, 19.78)

Monto respecto del tiempo

Máximo monto respecto de segundos

In [ ]:
monto_total = tiempo_fraudulento.map(lambda x: (1, x.ammount)).reduceByKey(lambda a,b: a + b).collect()[0][1]
(tiempo_fraudulento.map(lambda x: segundos(x.timestamp)).reduce(lambda a,b: a if a > b else b), redondeo(monto_total, 2), 100)

(13064705, 2614236.33, 100)

Top ten monto respecto de minutos

In [ ]:
porcentaje_lista(tiempo_fraudulento.map(lambda x: (minutos(x.timestamp), x.ammount)).reduceByKey(lambda a,b: a + b).takeOrdered(10, lambda x: -x[1]), monto_total)

[(176918, 6239.6, 0.2386),
 (99097, 4191.0, 0.1603),
 (199979, 3822.95, 0.1462),
 (189819, 3280.05, 0.1254),
 (87835, 3247.16, 0.1242),
 (120792, 3221.0, 0.1232),
 (151504, 3191.0, 0.122),
 (89633, 3190.0, 0.122),
 (151593, 3190.0, 0.122),
 (181769, 3190.0, 0.122)]

Last ten monto respecto de minutos

In [ ]:
porcentaje_lista(tiempo_fraudulento.map(lambda x: (minutos(x.timestamp), x.ammount)).reduceByKey(lambda a,b: a + b).takeOrdered(10, lambda x: x[1]), monto_total)

[(77470, 0.29, 0.0),
 (77302, 0.29, 0.0),
 (77307, 0.29, 0.0),
 (11188, 0.29, 0.0),
 (77311, 0.35, 0.0),
 (15646, 0.42, 0.0),
 (15588, 0.42, 0.0),
 (15594, 0.42, 0.0),
 (15639, 0.42, 0.0),
 (8390, 0.46, 0.0)]

Top ten monto respecto de horas

In [ ]:
porcentaje_lista(tiempo_fraudulento.map(lambda x: (horas(x.timestamp), x.ammount)).reduceByKey(lambda a,b: a + b).takeOrdered(10, lambda x: -x[1]), monto_total)

[(3000, 10744.97, 0.411),
 (2949, 10068.41, 0.3851),
 (3217, 9004.46, 0.3444),
 (2086, 6795.08, 0.2599),
 (2875, 6710.91, 0.2567),
 (2991, 6565.0, 0.2511),
 (2973, 6307.91, 0.2412),
 (2879, 6088.46, 0.2328),
 (3178, 6031.0, 0.2306),
 (3384, 5995.85, 0.2293)]

Last ten monto respecto de horas

In [ ]:
porcentaje_lista(tiempo_fraudulento.map(lambda x: (horas(x.timestamp), x.ammount)).reduceByKey(lambda a,b: a + b).takeOrdered(10, lambda x: x[1]), monto_total)

[(310, 2.44, 0.0),
 (3057, 6.16, 0.0002),
 (3297, 7.65, 0.0002),
 (2477, 8.21, 0.0003),
 (3588, 8.91, 0.0003),
 (2334, 9.54, 0.0003),
 (2433, 9.99, 0.0003),
 (2309, 10.35, 0.0003),
 (3441, 11.06, 0.0004),
 (2313, 12.68, 0.0004)]

Monto respecto de días

In [ ]:
porcentaje_lista(sorted(tiempo_fraudulento.map(lambda x: (dias(x.timestamp), x.ammount)).reduceByKey(lambda a,b: a + b).collect()), monto_total)

[(1, 20337.14, 0.7779),
 (2, 15359.6, 0.5875),
 (3, 14254.33, 0.5452),
 (4, 14120.44, 0.5401),
 (5, 16692.72, 0.6385),
 (6, 12869.08, 0.4922),
 (7, 21670.36, 0.8289),
 (8, 13508.15, 0.5167),
 (9, 13106.59, 0.5013),
 (10, 12709.72, 0.4861),
 (11, 21725.53, 0.831),
 (12, 14710.55, 0.5627),
 (13, 10975.38, 0.4198),
 (14, 15782.4, 0.6037),
 (15, 16438.68, 0.6288),
 (16, 7279.53, 0.2784),
 (17, 14338.32, 0.5484),
 (18, 19745.87, 0.7553),
 (19, 13647.86, 0.522),
 (20, 20877.6, 0.7986),
 (21, 11047.98, 0.4226),
 (22, 18677.98, 0.7144),
 (23, 15191.02, 0.581),
 (24, 11902.08, 0.4552),
 (25, 5965.49, 0.2281),
 (26, 12609.16, 0.4823),
 (27, 14725.04, 0.5632),
 (28, 22367.33, 0.8555),
 (29, 16953.83, 0.6485),
 (30, 22107.92, 0.8456),
 (31, 18823.93, 0.72),
 (32, 6477.42, 0.2477),
 (33, 18623.23, 0.7123),
 (34, 22765.2, 0.8708),
 (35, 15812.87, 0.6048),
 (36, 20959.42, 0.8017),
 (37, 24458.92, 0.9356),
 (38, 13548.03, 0.5182),
 (39, 15022.65, 0.5746),
 (40, 16653.4, 0.637),
 (41, 18989.12, 0.7263)

Top ten monto respecto de días

In [ ]:
porcentaje_lista(tiempo_fraudulento.map(lambda x: (dias(x.timestamp), x.ammount)).reduceByKey(lambda a,b: a + b).takeOrdered(10, lambda x: -x[1]), monto_total)

[(120, 39055.2, 1.4939),
 (125, 34599.42, 1.3235),
 (109, 32184.5, 1.2311),
 (95, 31478.14, 1.2041),
 (106, 30852.04, 1.1801),
 (114, 29092.98, 1.1128),
 (85, 28536.53, 1.0915),
 (64, 27517.51, 1.0526),
 (112, 27171.11, 1.0393),
 (132, 25954.06, 0.9927)]

Last ten monto respecto de días

In [ ]:
porcentaje_lista(tiempo_fraudulento.map(lambda x: (dias(x.timestamp), x.ammount)).reduceByKey(lambda a,b: a + b).takeOrdered(10, lambda x: x[1]), monto_total)

[(25, 5965.49, 0.2281),
 (32, 6477.42, 0.2477),
 (16, 7279.53, 0.2784),
 (152, 7747.41, 0.2963),
 (150, 7782.21, 0.2976),
 (149, 7826.8, 0.2993),
 (128, 7897.82, 0.3021),
 (67, 8018.99, 0.3067),
 (145, 8080.86, 0.3091),
 (80, 8365.21, 0.3199)]

Monto respecto de semanas

In [ ]:
porcentaje_lista(sorted(tiempo_fraudulento.map(lambda x: (semanas(x.timestamp), x.ammount)).reduceByKey(lambda a,b: a + b).collect()), monto_total)

[(1, 115303.69, 4.4106),
 (2, 102518.35, 3.9215),
 (3, 103375.85, 3.9543),
 (4, 101438.13, 3.8802),
 (5, 121564.43, 4.65),
 (6, 122015.66, 4.6673),
 (7, 104885.8, 4.0121),
 (8, 118201.74, 4.5214),
 (9, 127460.33, 4.8756),
 (10, 134119.43, 5.1303),
 (11, 108111.65, 4.1354),
 (12, 92534.23, 3.5396),
 (13, 147655.6, 5.6481),
 (14, 146352.54, 5.5982),
 (15, 123198.73, 4.7126),
 (16, 167064.38, 6.3905),
 (17, 158979.25, 6.0812),
 (18, 157838.82, 6.0376),
 (19, 114772.7, 4.3902),
 (20, 105325.59, 4.0289),
 (21, 94819.16, 3.627),
 (22, 46700.17, 1.7863)]

Top ten monto respecto de semanas

In [ ]:
porcentaje_lista(tiempo_fraudulento.map(lambda x: (semanas(x.timestamp), x.ammount)).reduceByKey(lambda a,b: a + b).takeOrdered(10, lambda x: -x[1]), monto_total)

[(16, 167064.38, 6.3905),
 (17, 158979.25, 6.0812),
 (18, 157838.82, 6.0376),
 (13, 147655.6, 5.6481),
 (14, 146352.54, 5.5982),
 (10, 134119.43, 5.1303),
 (9, 127460.33, 4.8756),
 (15, 123198.73, 4.7126),
 (6, 122015.66, 4.6673),
 (5, 121564.43, 4.65)]

Last ten monto respecto de semanas

In [ ]:
porcentaje_lista(tiempo_fraudulento.map(lambda x: (semanas(x.timestamp), x.ammount)).reduceByKey(lambda a,b: a + b).takeOrdered(10, lambda x: x[1]), monto_total)

[(22, 46700.17, 1.7863),
 (12, 92534.23, 3.5396),
 (21, 94819.16, 3.627),
 (4, 101438.13, 3.8802),
 (2, 102518.35, 3.9215),
 (3, 103375.85, 3.9543),
 (7, 104885.8, 4.0121),
 (20, 105325.59, 4.0289),
 (11, 108111.65, 4.1354),
 (19, 114772.7, 4.3902)]

Monto respecto de meses

In [ ]:
porcentaje_lista(sorted(tiempo_fraudulento.map(lambda x: (meses(x.timestamp), x.ammount)).reduceByKey(lambda a,b: a + b).collect()), monto_total)

[(1, 422636.04, 16.1667),
 (2, 466667.65, 17.851),
 (3, 462225.65, 17.681),
 (4, 584271.26, 22.3495),
 (5, 536916.38, 20.5381),
 (6, 141519.33, 5.4134)]

Máximo monto respecto de meses

In [ ]:
porcentaje_tupla(tiempo_fraudulento.map(lambda x: (meses(x.timestamp), x.ammount)).reduceByKey(lambda a,b: a + b).reduce(lambda a,b: a if a[1] > b[1] else b), monto_total)

(4, 584271.26, 22.34)

Plataformas fraudulentas

In [131]:
identity_rdd = sqlContext.read.csv('identity_features_train.csv', header=True, inferSchema=True).rdd

In [132]:
def unicidad_sistema(string) -> str:
    if string == None:
      return "NONE"
    lista = ["WINDOWS", "MOTO", "SAMSUNG", "LG", "SM", "GT", "RV", "ZTE", "HUAMWI", "IOS DEVICE","MACOS", "NOKIA", "ALCATEL", "IPHONE" ,"BUILD"]
    for sistema in lista:
        match string.upper().find(sistema):
            case -1:
                continue
            case _:
                return sistema
    return "OTRO"

def unicidad_modelo(string) -> str:
    if string == None:
      return "NONE"
    lista = ["CHROME", "SAFARI", "FIREFOX", "SAMSUNG", "OPERA", "EDGE", "ANDROID"]
    for modelo in lista:
        match string.upper().find(modelo):
            case -1:
                continue
            case _:
                return modelo
    return "OTRO"

In [133]:
identity_rdd.first()

Row(_c0=0, Id=2987004, feature01=0.0, feature02=70787.0, feature03=None, feature04=None, feature05=None, feature06=None, feature07=None, feature08=None, feature09=None, feature10=None, feature11=100.0, feature12='NotFound', feature13=None, feature14=-480.0, feature15='New', feature16='NotFound', feature17=166.0, feature18=None, feature19=542.0, feature20=144.0, feature21=None, feature22=None, feature23=None, feature24=None, feature25=None, feature26=None, feature27=None, feature28='New', feature29='NotFound', feature30='Android 7.0', feature31='samsung browser 6.2', feature32=32.0, feature33='2220x1080', feature34='match_status:2', feature35='T', feature36='F', feature37='T', feature38='T', Device='mobile', DeviceMetadata='SAMSUNG SM-G892A Build/NRD90M')

In [134]:
plataformas = identity_rdd.map(lambda x: (x.Id, (x.Device, x.DeviceMetadata, x.feature31))).leftOuterJoin(transactions_rdd.map(lambda x: (x.Id, x.target)))

In [135]:
plataformas.first()

(2987040, (('desktop', 'Windows', 'chrome 62.0'), 0))

Fraude por dispositivo

In [136]:
dispositivos = plataformas.map(lambda x: ((x[1][0][0], x[1][1]), 1)).reduceByKey(lambda a,b: a+b).map(lambda x: (x[0][0], (x[0][1], x[1])))

In [137]:
dispositivos_total = dispositivos.map(lambda x: (x[0], x[1][1])).reduceByKey(lambda a,b: a + b)

In [138]:
dispositivos = dispositivos.join(dispositivos_total).map(lambda x: (x[0], x[1][0][0], x[1][0][1], redondeo(x[1][0][1] * 100 / x[1][1], 2))).cache()

In [139]:
dispositivos.collect()

[(None, 0, 2866, 97.58),
 (None, 1, 71, 2.41),
 ('mobile', 0, 43198, 90.13),
 ('mobile', 1, 4726, 9.86),
 ('desktop', 0, 69946, 93.63),
 ('desktop', 1, 4758, 6.36)]

Dispositivo fraudulentos respecto de la cantidad

In [140]:
dispositivos.filter(lambda x: x[1] == 1).map(lambda x: (x[0], x[2])).sortBy(lambda x: -x[1]).collect()

[('desktop', 4758), ('mobile', 4726), (None, 71)]

Dispositivo legales respecto de la cantidad

In [141]:
dispositivos.filter(lambda x: x[1] == 0).map(lambda x: (x[0], x[2])).sortBy(lambda x: -x[1]).collect()

[('desktop', 69946), ('mobile', 43198), (None, 2866)]

Dispositivo fraudulentos respecto del promedio

In [142]:
dispositivos.filter(lambda x: x[1] == 1).map(lambda x: (x[0], x[3])).sortBy(lambda x: -x[1]).collect()

[('mobile', 9.86), ('desktop', 6.36), (None, 2.41)]

Dispositivo legales respecto del promedio

In [143]:
dispositivos.filter(lambda x: x[1] == 0).map(lambda x: (x[0], x[3])).sortBy(lambda x: -x[1]).collect()

[(None, 97.58), ('desktop', 93.63), ('mobile', 90.13)]

Fraude por sistema

In [144]:
sistema = plataformas.map(lambda x: ((unicidad_sistema(x[1][0][1]), x[1][1]), 1)).reduceByKey(lambda a,b: a+b).map(lambda x: (x[0][0], (x[0][1], x[1])))

In [145]:
sistema_total = sistema.map(lambda x: (x[0], x[1][1])).reduceByKey(lambda a,b: a + b)

In [146]:
sistema = sistema.join(sistema_total).map(lambda x: (x[0], x[1][0][0], x[1][0][1], redondeo(x[1][0][1] * 100 / x[1][1], 2))).cache()

In [147]:
sistema.collect()

[('BUILD', 0, 5233, 83.48),
 ('BUILD', 1, 1035, 16.51),
 ('MACOS', 0, 11054, 98.06),
 ('MACOS', 1, 218, 1.93),
 ('MOTO', 0, 2021, 84.27),
 ('MOTO', 1, 377, 15.72),
 ('RV', 0, 3673, 92.51),
 ('RV', 1, 297, 7.48),
 ('WINDOWS', 0, 38754, 93.69),
 ('WINDOWS', 1, 2610, 6.3),
 ('NOKIA', 1, 10, 100.0),
 ('IOS DEVICE', 0, 16571, 93.88),
 ('IOS DEVICE', 1, 1079, 6.11),
 ('ALCATEL', 0, 46, 86.79),
 ('ALCATEL', 1, 7, 13.2),
 ('SAMSUNG', 0, 2127, 92.11),
 ('SAMSUNG', 1, 182, 7.88),
 ('GT', 0, 113, 84.96),
 ('GT', 1, 20, 15.03),
 ('NONE', 1, 2330, 10.59),
 ('NONE', 0, 19660, 89.4),
 ('OTRO', 1, 154, 1.92),
 ('OTRO', 0, 7859, 98.07),
 ('ZTE', 0, 36, 80.0),
 ('ZTE', 1, 9, 20.0),
 ('SM', 1, 984, 12.47),
 ('SM', 0, 6901, 87.52),
 ('LG', 1, 242, 10.98),
 ('LG', 0, 1962, 89.01),
 ('IPHONE', 1, 1, 100.0)]

Top five sistema fraudulentos respecto de la cantidad

In [148]:
sistema.filter(lambda x: x[1] == 1).map(lambda x: (x[0], x[2])).takeOrdered(5, lambda x: -x[1])

[('WINDOWS', 2610),
 ('NONE', 2330),
 ('IOS DEVICE', 1079),
 ('BUILD', 1035),
 ('SM', 984)]

Last five sistema fraudulentos respecto de la cantidad

In [149]:
sistema.filter(lambda x: x[1] == 1).map(lambda x: (x[0], x[2])).takeOrdered(5, lambda x: x[1])

[('IPHONE', 1), ('ALCATEL', 7), ('ZTE', 9), ('NOKIA', 10), ('GT', 20)]

Top five sistema legales respecto de la cantidad

In [150]:
sistema.filter(lambda x: x[1] == 0).map(lambda x: (x[0], x[2])).takeOrdered(5, lambda x: -x[1])

[('WINDOWS', 38754),
 ('NONE', 19660),
 ('IOS DEVICE', 16571),
 ('MACOS', 11054),
 ('OTRO', 7859)]

Last five sistema legales respecto de la cantidad

In [151]:
sistema.filter(lambda x: x[1] == 0).map(lambda x: (x[0], x[2])).takeOrdered(5, lambda x: x[1])

[('ZTE', 36), ('ALCATEL', 46), ('GT', 113), ('LG', 1962), ('MOTO', 2021)]

Top five sistema fraudulento respecto del promedio

In [152]:
sistema.filter(lambda x: x[1] == 1).map(lambda x: (x[0], x[3])).takeOrdered(5, lambda x: -x[1])

[('NOKIA', 100.0),
 ('IPHONE', 100.0),
 ('ZTE', 20.0),
 ('BUILD', 16.51),
 ('MOTO', 15.72)]

Justificacion del promedio de cantidades bajas (El 100% se debe a que son solamente fraudulentos)

In [153]:
sistema.filter(lambda x: x[1] == 1).map(lambda x: (x[0], x[2], x[3])).takeOrdered(5, lambda x: -x[2])

[('NOKIA', 10, 100.0),
 ('IPHONE', 1, 100.0),
 ('ZTE', 9, 20.0),
 ('BUILD', 1035, 16.51),
 ('MOTO', 377, 15.72)]

Top five sistema fraudulento respecto de promedio filtrado por cantidad menor a 100

In [154]:
sistema.filter(lambda x: x[1] == 1 and x[2] < 100).map(lambda x: (x[0], x[2], x[3])).takeOrdered(5, lambda x: -x[2])

[('NOKIA', 10, 100.0),
 ('IPHONE', 1, 100.0),
 ('ZTE', 9, 20.0),
 ('GT', 20, 15.03),
 ('ALCATEL', 7, 13.2)]

Top five sistema fraudulento respecto de promedio filtrado por cantidad mayor a 100

In [155]:
sistema.filter(lambda x: x[1] == 1 and x[2] > 100).map(lambda x: (x[0], x[2], x[3])).takeOrdered(5, lambda x: -x[2])

[('BUILD', 1035, 16.51),
 ('MOTO', 377, 15.72),
 ('SM', 984, 12.47),
 ('LG', 242, 10.98),
 ('NONE', 2330, 10.59)]

Last five sistema fraudulento respecto del promedio

In [156]:
sistema.filter(lambda x: x[1] == 1).map(lambda x: (x[0], x[3])).takeOrdered(5, lambda x: x[1])

[('OTRO', 1.92),
 ('MACOS', 1.93),
 ('IOS DEVICE', 6.11),
 ('WINDOWS', 6.3),
 ('RV', 7.48)]

Top five sistema legal respecto del promedio

In [157]:
sistema.filter(lambda x: x[1] == 0).map(lambda x: (x[0], x[3])).takeOrdered(5, lambda x: -x[1])

[('OTRO', 98.07),
 ('MACOS', 98.06),
 ('IOS DEVICE', 93.88),
 ('WINDOWS', 93.69),
 ('RV', 92.51)]

Last five sistema legal respecto del promedio

In [158]:
sistema.filter(lambda x: x[1] == 0).map(lambda x: (x[0], x[3])).takeOrdered(5, lambda x: x[1])

[('ZTE', 80.0),
 ('BUILD', 83.48),
 ('MOTO', 84.27),
 ('GT', 84.96),
 ('ALCATEL', 86.79)]

Fraude por modelo

In [159]:
modelo = plataformas.map(lambda x: ((unicidad_modelo(x[1][0][2]), x[1][1]), 1)).reduceByKey(lambda a,b: a+b).map(lambda x: (x[0][0], (x[0][1], x[1])))

In [160]:
modelo_total = modelo.map(lambda x: (x[0], x[1][1])).reduceByKey(lambda a,b: a + b)

In [161]:
modelo = modelo.join(modelo_total).map(lambda x: (x[0], x[1][0][0], x[1][0][1], redondeo(x[1][0][1] * 100 / x[1][1], 2))).cache()

In [162]:
modelo.collect()

[('ANDROID', 0, 358, 79.55),
 ('ANDROID', 1, 92, 20.44),
 ('FIREFOX', 0, 5830, 92.53),
 ('FIREFOX', 1, 470, 7.46),
 ('SAMSUNG', 0, 1758, 91.75),
 ('SAMSUNG', 1, 158, 8.24),
 ('NONE', 1, 88, 2.81),
 ('NONE', 0, 3037, 97.18),
 ('OTRO', 1, 231, 2.51),
 ('OTRO', 0, 8964, 97.48),
 ('CHROME', 0, 60081, 90.7),
 ('CHROME', 1, 6158, 9.29),
 ('EDGE', 0, 4624, 96.31),
 ('EDGE', 1, 177, 3.68),
 ('SAFARI', 0, 31151, 93.81),
 ('SAFARI', 1, 2052, 6.18),
 ('OPERA', 0, 207, 61.6),
 ('OPERA', 1, 129, 38.39)]

Top three modelo fraudulento respecto de la cantidad

In [163]:
modelo.filter(lambda x: x[1] == 1).map(lambda x: (x[0], x[2])).takeOrdered(3, lambda x: -x[1])

[('CHROME', 6158), ('SAFARI', 2052), ('FIREFOX', 470)]

Last three modelo fraudulento respecto de la cantidad

In [164]:
modelo.filter(lambda x: x[1] == 1).map(lambda x: (x[0], x[2])).takeOrdered(3, lambda x: x[1])

[('NONE', 88), ('ANDROID', 92), ('OPERA', 129)]

Top three modelo legal respecto de la cantidad

In [165]:
modelo.filter(lambda x: x[1] == 0).map(lambda x: (x[0], x[2])).takeOrdered(3, lambda x: -x[1])

[('CHROME', 60081), ('SAFARI', 31151), ('OTRO', 8964)]

Last three modelo legal respecto de la cantidad

In [166]:
modelo.filter(lambda x: x[1] == 0).map(lambda x: (x[0], x[2])).takeOrdered(3, lambda x: x[1])

[('OPERA', 207), ('ANDROID', 358), ('SAMSUNG', 1758)]

Top three modelo fraudulento respecto del promedio

In [167]:
modelo.filter(lambda x: x[1] == 1).map(lambda x: (x[0], x[3])).takeOrdered(3, lambda x: -x[1])

[('OPERA', 38.39), ('ANDROID', 20.44), ('CHROME', 9.29)]

Last three modelo fraudulento respecto del promedio

In [168]:
modelo.filter(lambda x: x[1] == 1).map(lambda x: (x[0], x[3])).takeOrdered(3, lambda x: x[1])

[('OTRO', 2.51), ('NONE', 2.81), ('EDGE', 3.68)]

Top three modelo legal respecto del promedio

In [169]:
modelo.filter(lambda x: x[1] == 0).map(lambda x: (x[0], x[3])).takeOrdered(3, lambda x: -x[1])

[('OTRO', 97.48), ('NONE', 97.18), ('EDGE', 96.31)]

Last three modelo legal respecto del promedio

In [170]:
modelo.filter(lambda x: x[1] == 1).map(lambda x: (x[0], x[3])).takeOrdered(3, lambda x: x[1])

[('OTRO', 2.51), ('NONE', 2.81), ('EDGE', 3.68)]

Fraudes de tarjetas

In [171]:
tarjetas = transactions_rdd.map(lambda x: ((x.feat_card_4, x.target), 1)).reduceByKey(lambda a,b: a + b).map(lambda x: (x[0][0], (x[0][1], x[1])))

In [172]:
tarjetas_total = tarjetas.map(lambda x: (x[0], x[1][1])).reduceByKey(lambda a,b: a + b)

In [173]:
tarjetas = tarjetas.join(tarjetas_total).map(lambda x: (x[0], x[1][0][0], x[1][0][1], redondeo(x[1][0][1] * 100 / x[1][1], 2))).cache()

In [174]:
tarjetas.collect()

[(None, 1, 26, 3.12),
 (None, 0, 807, 96.87),
 ('discover', 0, 5291, 92.4),
 ('discover', 1, 435, 7.59),
 ('american express', 1, 218, 2.9),
 ('american express', 0, 7274, 97.09),
 ('visa', 1, 11327, 3.45),
 ('visa', 0, 316498, 96.54),
 ('mastercard', 0, 154509, 96.51),
 ('mastercard', 1, 5574, 3.48)]

Cantidad legal de tarjeta

In [175]:
tarjetas.filter(lambda x: x[1] == 1).map(lambda x: (x[0], x[2])).takeOrdered(5, lambda x: -x[1])

[('visa', 11327),
 ('mastercard', 5574),
 ('discover', 435),
 ('american express', 218),
 (None, 26)]

Cantidad legal de tarjeta

In [176]:
tarjetas.filter(lambda x: x[1] == 0).map(lambda x: (x[0], x[2])).takeOrdered(5, lambda x: -x[1])

[('visa', 316498),
 ('mastercard', 154509),
 ('american express', 7274),
 ('discover', 5291),
 (None, 807)]

Promedio fraudulento de tarjeta

In [177]:
tarjetas.filter(lambda x: x[1] == 1).map(lambda x: (x[0], x[3])).takeOrdered(5, lambda x: -x[1])

[('discover', 7.59),
 ('mastercard', 3.48),
 ('visa', 3.45),
 (None, 3.12),
 ('american express', 2.9)]

Promedio legal de tarjeta

In [178]:
tarjetas.filter(lambda x: x[1] == 0).map(lambda x: (x[0], x[3])).takeOrdered(5, lambda x: -x[1])

[('american express', 97.09),
 (None, 96.87),
 ('visa', 96.54),
 ('mastercard', 96.51),
 ('discover', 92.4)]

Fraude por tipo de pago

In [179]:
banco = transactions_rdd.map(lambda x: ((x.feat_card_6, x.target), 1)).reduceByKey(lambda a,b: a + b).map(lambda x: (x[0][0], (x[0][1], x[1])))

In [180]:
banco_total = banco.map(lambda x: (x[0], x[1][1])).reduceByKey(lambda a,b: a + b)

In [181]:
banco = banco.join(banco_total).map(lambda x: (x[0], x[1][0][0], x[1][0][1], redondeo(x[1][0][1] * 100 / x[1][1], 2))).cache()

In [182]:
banco.collect()

[('charge card', 0, 15, 100.0),
 (None, 1, 24, 2.89),
 (None, 0, 804, 97.1),
 ('credit', 1, 8626, 6.72),
 ('credit', 0, 119575, 93.27),
 ('debit', 1, 8930, 2.39),
 ('debit', 0, 363955, 97.6),
 ('debit or credit', 0, 30, 100.0)]

Máxima cantidad de tipo de pago fraudulento

In [183]:
banco.filter(lambda x: x[1] == 1).map(lambda x: (x[0], x[2])).sortBy(lambda x: -x[1]).collect()

[('debit', 8930), ('credit', 8626), (None, 24)]

Máxima cantidad de tipo de pago legal

In [184]:
banco.filter(lambda x: x[1] == 0).map(lambda x: (x[0], x[2])).sortBy(lambda x: -x[1]).collect()

[('debit', 363955),
 ('credit', 119575),
 (None, 804),
 ('debit or credit', 30),
 ('charge card', 15)]

Máximo promodio por tipo de pago fraudulento

In [185]:
banco.filter(lambda x: x[1] == 1).map(lambda x: (x[0], x[3])).sortBy(lambda x: -x[1]).collect()

[('credit', 6.72), (None, 2.89), ('debit', 2.39)]

Máximo promodio por tipo de pago legal (El 100% se debe a que son solamente legales)

In [186]:
banco.filter(lambda x: x[1] == 0).map(lambda x: (x[0], x[3])).sortBy(lambda x: -x[1]).collect()

[('charge card', 100.0),
 ('debit or credit', 100.0),
 ('debit', 97.6),
 (None, 97.1),
 ('credit', 93.27)]

Fraude segun el correo

In [187]:
def unicidad_correo(string) -> str:
  if string == None:
    return "none"
  lista = ["yahoo", "gmail", "hotmail", "outlook", "live"]
  for correo in lista:
    match string.lower().find(correo):
      case -1:
        continue
      case _:
        return f"{correo}.com"
  return string

Correo comprador

In [188]:
correo_comprador = transactions_rdd.map(lambda x: ((unicidad_correo(x.payer_email_domain), x.target), 1)).reduceByKey(lambda a,b: a + b)\
.map(lambda x: (x[0][0], (x[0][1], x[1])))

In [189]:
correo_comprador_total = correo_comprador.map(lambda x: (x[0], x[1][1])).reduceByKey(lambda a,b: a + b)

In [190]:
correo_comprador = correo_comprador.join(correo_comprador_total).map(lambda x: (x[0], x[1][0][0], x[1][0][1], redondeo(x[1][0][1] * 100 / x[1][1], 2))).cache()

In [191]:
correo_comprador.collect()

[('cfl.rr.com', 0, 142, 100.0),
 ('msn.com', 1, 62, 1.76),
 ('msn.com', 0, 3445, 98.23),
 ('centurylink.net', 0, 174, 100.0),
 ('none', 1, 2346, 2.99),
 ('none', 0, 76080, 97.0),
 ('q.com', 0, 162, 100.0),
 ('mac.com', 0, 365, 97.07),
 ('mac.com', 1, 11, 2.92),
 ('embarqmail.com', 1, 7, 3.08),
 ('embarqmail.com', 0, 220, 96.91),
 ('bellsouth.net', 0, 1639, 97.21),
 ('bellsouth.net', 1, 47, 2.78),
 ('ptd.net', 0, 62, 100.0),
 ('suddenlink.net', 1, 4, 3.27),
 ('suddenlink.net', 0, 118, 96.72),
 ('protonmail.com', 1, 31, 45.58),
 ('protonmail.com', 0, 37, 54.41),
 ('optonline.net', 1, 17, 1.93),
 ('optonline.net', 0, 863, 98.06),
 ('servicios-ta.com', 0, 28, 100.0),
 ('outlook.com', 0, 4256, 90.51),
 ('outlook.com', 1, 446, 9.48),
 ('anonymous.com', 0, 31024, 97.69),
 ('anonymous.com', 1, 731, 2.3),
 ('icloud.com', 0, 5146, 96.76),
 ('icloud.com', 1, 172, 3.23),
 ('frontier.com', 0, 241, 99.58),
 ('frontier.com', 1, 1, 0.41),
 ('aol.com', 0, 23522, 97.73),
 ('aol.com', 1, 545, 2.26),
 ('y

Top ten cantidad de correo fraudulento

In [192]:
correo_comprador.filter(lambda x: x[1] == 1).map(lambda x: (x[0], x[2])).takeOrdered(10, lambda x: -x[1])

[('gmail.com', 8511),
 ('none', 2346),
 ('hotmail.com', 2038),
 ('yahoo.com', 1989),
 ('anonymous.com', 731),
 ('aol.com', 545),
 ('outlook.com', 446),
 ('comcast.net', 203),
 ('icloud.com', 172),
 ('live.com', 103)]

Last ten cantidad de correo fraudulento

In [193]:
correo_comprador.filter(lambda x: x[1] == 1).map(lambda x: (x[0], x[2])).takeOrdered(10, lambda x: x[1])

[('frontier.com', 1),
 ('sc.rr.com', 1),
 ('netzero.net', 1),
 ('prodigy.net.mx', 1),
 ('rocketmail.com', 2),
 ('roadrunner.com', 3),
 ('cableone.net', 3),
 ('suddenlink.net', 4),
 ('frontiernet.net', 5),
 ('earthlink.net', 6)]

Top ten cantidad de correo legal

In [194]:
correo_comprador.filter(lambda x: x[1] == 0).map(lambda x: (x[0], x[2])).takeOrdered(10, lambda x: -x[1])

[('gmail.com', 186133),
 ('yahoo.com', 85789),
 ('none', 76080),
 ('hotmail.com', 37730),
 ('anonymous.com', 31024),
 ('aol.com', 23522),
 ('comcast.net', 6677),
 ('icloud.com', 5146),
 ('outlook.com', 4256),
 ('msn.com', 3445)]

Last ten cantidad de correo legal

In [195]:
correo_comprador.filter(lambda x: x[1] == 0).map(lambda x: (x[0], x[2])).takeOrdered(10, lambda x: x[1])

[('servicios-ta.com', 28),
 ('protonmail.com', 37),
 ('ptd.net', 62),
 ('suddenlink.net', 118),
 ('gmx.de', 124),
 ('sc.rr.com', 135),
 ('cableone.net', 135),
 ('cfl.rr.com', 142),
 ('frontiernet.net', 158),
 ('q.com', 162)]

Top ten promedio de correo fraudulento

In [196]:
correo_comprador.filter(lambda x: x[1] == 1).map(lambda x: (x[0], x[3])).takeOrdered(10, lambda x: -x[1])

[('protonmail.com', 45.58),
 ('mail.com', 19.95),
 ('aim.com', 13.98),
 ('outlook.com', 9.48),
 ('hotmail.com', 5.12),
 ('gmail.com', 4.37),
 ('suddenlink.net', 3.27),
 ('icloud.com', 3.23),
 ('live.com', 3.13),
 ('embarqmail.com', 3.08)]

Last ten promedio de correo fraudulento

In [197]:
correo_comprador.filter(lambda x: x[1] == 1).map(lambda x: (x[0], x[3])).takeOrdered(10, lambda x: x[1])

[('rocketmail.com', 0.35),
 ('frontier.com', 0.41),
 ('sbcglobal.net', 0.42),
 ('prodigy.net.mx', 0.54),
 ('netzero.net', 0.58),
 ('verizon.net', 0.63),
 ('att.net', 0.65),
 ('sc.rr.com', 0.73),
 ('roadrunner.com', 1.17),
 ('earthlink.net', 1.36)]

Top ten promedio de correo legal (El 100% se debe a que son solamente legales)

In [198]:
correo_comprador.filter(lambda x: x[1] == 0).map(lambda x: (x[0], x[3])).takeOrdered(11, lambda x: -x[1])

[('cfl.rr.com', 100.0),
 ('centurylink.net', 100.0),
 ('q.com', 100.0),
 ('ptd.net', 100.0),
 ('servicios-ta.com', 100.0),
 ('twc.com', 100.0),
 ('windstream.net', 100.0),
 ('netzero.com', 100.0),
 ('web.de', 100.0),
 ('gmx.de', 100.0),
 ('rocketmail.com', 99.64)]

Last ten promedio de correo legal

In [199]:
correo_comprador.filter(lambda x: x[1] == 0).map(lambda x: (x[0], x[3])).takeOrdered(10, lambda x: x[1])

[('protonmail.com', 54.41),
 ('mail.com', 80.04),
 ('aim.com', 86.01),
 ('outlook.com', 90.51),
 ('hotmail.com', 94.87),
 ('gmail.com', 95.62),
 ('suddenlink.net', 96.72),
 ('icloud.com', 96.76),
 ('live.com', 96.86),
 ('embarqmail.com', 96.91)]

Correo vendedor

In [200]:
correo_vendedor = transactions_rdd.map(lambda x: ((unicidad_correo(x.receiver_email_domain), x.target), 1)).reduceByKey(lambda a,b: a + b)\
.map(lambda x: (x[0][0], (x[0][1], x[1])))

In [201]:
correo_vendedor_total = correo_vendedor.map(lambda x: (x[0], x[1][1])).reduceByKey(lambda a,b: a + b)

In [202]:
correo_vendedor = correo_vendedor.join(correo_vendedor_total).map(lambda x: (x[0], x[1][0][0], x[1][0][1], redondeo(x[1][0][1] * 100 / x[1][1], 2))).cache()

In [203]:
correo_vendedor.collect()

[('cfl.rr.com', 0, 35, 100.0),
 ('msn.com', 1, 1, 0.13),
 ('msn.com', 0, 733, 99.86),
 ('centurylink.net', 0, 11, 100.0),
 ('none', 1, 8105, 2.11),
 ('none', 0, 375081, 97.88),
 ('q.com', 0, 24, 100.0),
 ('mac.com', 0, 195, 100.0),
 ('bellsouth.net', 0, 378, 100.0),
 ('ptd.net', 0, 27, 100.0),
 ('embarqmail.com', 0, 59, 100.0),
 ('scranton.edu', 0, 57, 100.0),
 ('suddenlink.net', 1, 1, 4.34),
 ('suddenlink.net', 0, 22, 95.65),
 ('protonmail.com', 1, 38, 95.0),
 ('protonmail.com', 0, 2, 5.0),
 ('optonline.net', 1, 2, 1.19),
 ('optonline.net', 0, 166, 98.8),
 ('servicios-ta.com', 0, 28, 100.0),
 ('outlook.com', 0, 2125, 84.59),
 ('outlook.com', 1, 387, 15.4),
 ('anonymous.com', 0, 17669, 97.19),
 ('anonymous.com', 1, 509, 2.8),
 ('icloud.com', 0, 1042, 87.41),
 ('icloud.com', 1, 150, 12.58),
 ('frontier.com', 0, 47, 100.0),
 ('aol.com', 0, 3240, 96.37),
 ('aol.com', 1, 122, 3.62),
 ('ymail.com', 1, 8, 4.32),
 ('ymail.com', 0, 177, 95.67),
 ('cox.net', 1, 6, 1.47),
 ('cox.net', 0, 401, 98

Top ten cantidad de correo fraudulento

In [204]:
correo_vendedor.filter(lambda x: x[1] == 1).map(lambda x: (x[0], x[2])).takeOrdered(10, lambda x: -x[1])

[('none', 8105),
 ('gmail.com', 5731),
 ('hotmail.com', 1812),
 ('yahoo.com', 564),
 ('anonymous.com', 509),
 ('outlook.com', 387),
 ('icloud.com', 150),
 ('aol.com', 122),
 ('live.com', 59),
 ('mail.com', 46)]

Last ten cantidad de correo fraudulento

In [205]:
correo_vendedor.filter(lambda x: x[1] == 1).map(lambda x: (x[0], x[2])).takeOrdered(10, lambda x: x[1])

[('msn.com', 1),
 ('suddenlink.net', 1),
 ('me.com', 1),
 ('sbcglobal.net', 1),
 ('aim.com', 1),
 ('netzero.net', 1),
 ('prodigy.net.mx', 1),
 ('optonline.net', 2),
 ('rocketmail.com', 2),
 ('charter.net', 5)]

Top ten cantidad de correo legal

In [206]:
correo_vendedor.filter(lambda x: x[1] == 0).map(lambda x: (x[0], x[2])).takeOrdered(10, lambda x: -x[1])

[('none', 375081),
 ('gmail.com', 43850),
 ('hotmail.com', 22825),
 ('anonymous.com', 17669),
 ('yahoo.com', 10268),
 ('aol.com', 3240),
 ('outlook.com', 2125),
 ('comcast.net', 1590),
 ('live.com', 1321),
 ('icloud.com', 1042)]

Last ten cantidad de correo legal

In [207]:
correo_vendedor.filter(lambda x: x[1] == 0).map(lambda x: (x[0], x[2])).takeOrdered(10, lambda x: x[1])

[('protonmail.com', 2),
 ('netzero.net', 7),
 ('sc.rr.com', 8),
 ('centurylink.net', 11),
 ('netzero.com', 13),
 ('frontiernet.net', 13),
 ('suddenlink.net', 22),
 ('q.com', 24),
 ('cableone.net', 24),
 ('twc.com', 25)]

Top ten promedio de correo fraudulento

In [208]:
correo_vendedor.filter(lambda x: x[1] == 1).map(lambda x: (x[0], x[3])).takeOrdered(10, lambda x: -x[1])

[('protonmail.com', 95.0),
 ('mail.com', 38.65),
 ('outlook.com', 15.4),
 ('icloud.com', 12.58),
 ('netzero.net', 12.5),
 ('gmail.com', 11.55),
 ('earthlink.net', 8.45),
 ('hotmail.com', 7.35),
 ('yahoo.com', 5.2),
 ('charter.net', 4.42)]

Last ten promedio de correo fraudulento

In [209]:
correo_vendedor.filter(lambda x: x[1] == 1).map(lambda x: (x[0], x[3])).takeOrdered(10, lambda x: x[1])

[('msn.com', 0.13),
 ('me.com', 0.2),
 ('sbcglobal.net', 0.2),
 ('prodigy.net.mx', 0.54),
 ('optonline.net', 1.19),
 ('comcast.net', 1.3),
 ('cox.net', 1.47),
 ('none', 2.11),
 ('anonymous.com', 2.8),
 ('aim.com', 3.12)]

Top ten promedio de correo legal

In [210]:
correo_vendedor.filter(lambda x: x[1] == 0).map(lambda x: (x[0], x[3])).takeOrdered(10, lambda x: -x[1])

[('cfl.rr.com', 100.0),
 ('centurylink.net', 100.0),
 ('q.com', 100.0),
 ('mac.com', 100.0),
 ('bellsouth.net', 100.0),
 ('ptd.net', 100.0),
 ('embarqmail.com', 100.0),
 ('scranton.edu', 100.0),
 ('servicios-ta.com', 100.0),
 ('frontier.com', 100.0)]

Cantidad de correos totalmente legales

In [211]:
correo_vendedor.filter(lambda x: x[1] == 0 and x[3] == 100).count()

22

Top ten promedio de correo legal menor al 100%

In [212]:
correo_vendedor.filter(lambda x: x[1] == 0 and x[3] < 100).map(lambda x: (x[0], x[3])).takeOrdered(10, lambda x: -x[1])

[('msn.com', 99.86),
 ('me.com', 99.79),
 ('sbcglobal.net', 99.79),
 ('prodigy.net.mx', 99.45),
 ('optonline.net', 98.8),
 ('comcast.net', 98.69),
 ('cox.net', 98.52),
 ('none', 97.88),
 ('anonymous.com', 97.19),
 ('aim.com', 96.87)]

Last ten promedio de correo legal

In [213]:
correo_vendedor.filter(lambda x: x[1] == 0).map(lambda x: (x[0], x[3])).takeOrdered(10, lambda x: x[1])

[('protonmail.com', 5.0),
 ('mail.com', 61.34),
 ('outlook.com', 84.59),
 ('icloud.com', 87.41),
 ('netzero.net', 87.5),
 ('gmail.com', 88.44),
 ('earthlink.net', 91.54),
 ('hotmail.com', 92.64),
 ('yahoo.com', 94.79),
 ('charter.net', 95.57)]

Sistema Operativo

In [214]:
def unicidad_sistema_operativo(string) -> str:
    if string == None:
      return "NONE"
    lista = ["IOS", "MAC", "WINDOWS", "ANDROID", "LINUX"]
    for sistema_operativo in lista:
        match string.upper().find(sistema_operativo):
            case -1:
                continue
            case _:
                return sistema_operativo
    return "OTRO"

In [215]:
sistema_operativo = identity_rdd.map(lambda x: (x.Id, x.feature30)).leftOuterJoin(transactions_rdd.map(lambda x: (x.Id, x.target)))\
.map(lambda x: ((unicidad_sistema_operativo(x[1][0]), x[1][1]), 1)).reduceByKey(lambda a,b: a + b).map(lambda x: (x[0][0], (x[0][1], x[1])))

In [216]:
sistema_operativo_total = sistema_operativo.map(lambda x: (x[0], x[1][1])).reduceByKey(lambda a,b: a + b)

In [217]:
sistema_operativo = sistema_operativo.join(sistema_operativo_total).map(lambda x: (x[0], x[1][0][0], x[1][0][1], redondeo(x[1][0][1] * 100 / x[1][1], 2))).cache()

In [218]:
sistema_operativo.collect()

[('ANDROID', 0, 5158, 91.56),
 ('ANDROID', 1, 475, 8.43),
 ('WINDOWS', 0, 30923, 96.48),
 ('WINDOWS', 1, 1128, 3.51),
 ('IOS', 0, 16571, 93.88),
 ('IOS', 1, 1079, 6.11),
 ('LINUX', 0, 905, 92.06),
 ('LINUX', 1, 78, 7.93),
 ('MAC', 0, 11969, 98.06),
 ('MAC', 1, 236, 1.93),
 ('NONE', 1, 6556, 11.49),
 ('NONE', 0, 50468, 88.5),
 ('OTRO', 1, 3, 15.78),
 ('OTRO', 0, 16, 84.21)]

Máxima cantidad de sistemas operativos fraudulentos

In [219]:
sistema_operativo.filter(lambda x: x[1] == 1).map(lambda x: (x[0], x[2])).sortBy(lambda x: -x[1]).collect()

[('NONE', 6556),
 ('WINDOWS', 1128),
 ('IOS', 1079),
 ('ANDROID', 475),
 ('MAC', 236),
 ('LINUX', 78),
 ('OTRO', 3)]

Máxima cantidad de sistemas operativos legales

In [220]:
sistema_operativo.filter(lambda x: x[1] == 0).map(lambda x: (x[0], x[2])).sortBy(lambda x: -x[1]).collect()

[('NONE', 50468),
 ('WINDOWS', 30923),
 ('IOS', 16571),
 ('MAC', 11969),
 ('ANDROID', 5158),
 ('LINUX', 905),
 ('OTRO', 16)]

Máximo promedio de sistemas operativos fraudulentos ('OTRO' no tiene mucha cantidad)

In [221]:
sistema_operativo.filter(lambda x: x[1] == 1).map(lambda x: (x[0], x[3])).sortBy(lambda x: -x[1]).collect()

[('OTRO', 15.78),
 ('NONE', 11.49),
 ('ANDROID', 8.43),
 ('LINUX', 7.93),
 ('IOS', 6.11),
 ('WINDOWS', 3.51),
 ('MAC', 1.93)]

Máximo promedio de sistemas operativos legales

In [222]:
sistema_operativo.filter(lambda x: x[1] == 0).map(lambda x: (x[0], x[3])).sortBy(lambda x: -x[1]).collect()

[('MAC', 98.06),
 ('WINDOWS', 96.48),
 ('IOS', 93.88),
 ('LINUX', 92.06),
 ('ANDROID', 91.56),
 ('NONE', 88.5),
 ('OTRO', 84.21)]